In [31]:
'查看供应关系终止的原因'
from API.ai_ask import ask_qwen
from Neo4jHost import get_reomte_driver
import json
from tqdm import tqdm

neo4j_host=get_reomte_driver()

In [2]:
stop_event=neo4j_host.execute_query("match (n)-[r:SupplyProductTo]->(m) where r.status='Stopped' return n,r,m")

In [32]:
import json
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor, as_completed

def process_record(record):
    supplier = record["n"].get("name", "")
    supplier_country = record["n"].get("country", "")
    original_content = record["r"].get("original_content", "")
    
    # Fix potential typo in key name
    if "process" in record["r"]:
        analysing_prcess = record["r"].get("process", "")
    elif "analysing_prcess" in record["r"]:
        analysing_prcess = record["r"].get("analysing_prcess", "")
    
    customer = record["m"].get("name", "")
    customer_country = record["m"].get("country", "")
    
    if original_content:
        ai_response = ask_qwen(
            prompt_text=f'''
                请看一段关于{supplier}和{customer}的文本，严格遵照文本，结合你的逻辑分析回答问题
                {original_content}
                1. 请根据它来回答你认为上述描述中是否存在供货关系终止的情况？请先给出你的分析，在IsShutDown_Analyzation处进行分析。如果存在，请在IsShutDown处填写True，如果不存在，请填写False
                2. 如果存在供货关系终止的情况，请在ReasonForShutDown_Analyzation处分析终止发生的原因，然后ReasonForShutDown处用一个短语概括这一原因。可以选择的原因短语包括：政策变化与国际制裁、自主研发或国产替代、二者出现竞争关系、经营战略调整、技术路线调整、法律纠纷、意外灾害、供应商质量不达标、供应商价格过高、供应商稳定性不足等等。从这些当中仅选择一个你认为最符合的。如果没有合适的，可以填写你认为更符合的。
                你需要以如下的json格式回答问题：
                {{
                    "IsShutDown_Analyzation": str,
                    "IsShutDown": bool,
                    "ReasonForShutDown_Analyzation": str,
                    "ReasonForShutDown": str
                }}
                务必基于原文
            ''',
            system_instruction="你是一个行业研究人员，擅长分析商业活动中的因果关系，用json回答问题",
            mode="json",
            model="qwen-max"
        )
        
        try:
            answer_dict = json.loads(ai_response)
            answer_dict.update({
                "supplier": supplier,
                "supplier_country": supplier_country,
                "customer": customer,
                "customer_country": customer_country,
                "original_content": original_content
            })
            return answer_dict
        except Exception as e:
            print(f"Error processing record: {e}")
            return None
    return None

def process_stop_events(stop_event):
    reason_for_stop = []
    
    with ThreadPoolExecutor(max_workers=10) as executor:  # Adjust max_workers as needed
        futures = [executor.submit(process_record, record) for record in stop_event]
        
        for future in tqdm(as_completed(futures), total=len(futures)):
            result = future.result()
            if result:
                reason_for_stop.append(result)
    
    return reason_for_stop

# Example usage
# stop_event = [...]  # Your input data here
result = process_stop_events(stop_event)
print(result)

 20%|█▉        | 349/1781 [05:01<30:41,  1.29s/it]

Error processing record: Unterminated string starting at: line 2 column 31 (char 32)


 27%|██▋       | 485/1781 [06:28<17:47,  1.21it/s]

gpt error Error code: 400 - {'error': {'code': 'data_inspection_failed', 'param': None, 'message': 'Output data may contain inappropriate content.', 'type': 'data_inspection_failed'}, 'id': 'chatcmpl-9ef86be7-21f6-9f95-ba56-478c36dcb2a0', 'request_id': '9ef86be7-21f6-9f95-ba56-478c36dcb2a0'}
Error processing record: the JSON object must be str, bytes or bytearray, not NoneType


 27%|██▋       | 488/1781 [06:29<13:53,  1.55it/s]

gpt error Error code: 400 - {'error': {'code': 'data_inspection_failed', 'param': None, 'message': 'Output data may contain inappropriate content.', 'type': 'data_inspection_failed'}, 'id': 'chatcmpl-510453fc-2681-98e3-838d-bb37d8e72da7', 'request_id': '510453fc-2681-98e3-838d-bb37d8e72da7'}
Error processing record: the JSON object must be str, bytes or bytearray, not NoneType


100%|██████████| 1781/1781 [21:02<00:00,  1.41it/s]

[{'IsShutDown_Analyzation': '文本中提到的案例主要涉及法律纠纷。在第一个案例中，福建兴大进出口贸易有限公司与山煤国际能源集团华南有限公司之间的争议是因为仓储保管问题，而非直接的供货关系终止。第二个案例中，厦门象屿物流集团有限责任公司与华泰重工（南通）有限公司之间存在的是出口代理协议相关的争议，并且提到了象屿物流向法院起诉要求支付代垫款项等费用，这表明双方的合作可能受到了严重影响，但并未明确说明供货关系终止。', 'IsShutDown': False, 'ReasonForShutDown_Analyzation': '', 'ReasonForShutDown': '', 'supplier': '厦门象屿物流集团有限责任公司', 'supplier_country': '中国大陆', 'customer': '华泰重工（南通）有限公司', 'customer_country': '中国大陆', 'original_content': '《<b>厦门象屿股份有限公司</b> - 上海证券交易所》: 案由：子公司福建兴大进出口贸易有限公司（以下简称“福建兴大”）将煤炭交由山煤国际\n能源集团华南有限公司（以下简称“山煤国际能源”）仓储保管，后保管单位拒绝交付相应货\n物。 \n进展：2016年8月，福建省高级人民法院一审判决山煤国际能源交付煤炭，若无法交付应\n赔偿损失 6,806万元及利息损失，北京亚欧经贸集团有限公司、汤龙硕、施天灵承担连带清偿责\n任。山煤国际能源已向最高人民法院提起上诉，最高人民法院裁定发回福建省高级人民法院重\n审。截至本报告日，该案一审审理中。  2020年年度报告  \n268 / 280 \n 财务影响：截至 2020年12月31日，本公司已计提坏账准备 2,780万元。 \n②：2018年1月，象屿物流向厦门海事法院起诉，要求华泰重工及其连带清偿责任担保方\n南通华凯重工有限公司、南通向隆船舶科技发展有限公司、香港华泰发展有限公司、霍起、徐小\n寓、朱红兵支付代垫款项、代垫款项利息、代理费等总计约 6,596万元。 \n案由：子公司厦门象屿物流集团有限责任公司（以下简称“象屿物流”）与华泰重工（南\n通）有限公 司（以下简称“华泰重工”）签订散货船出口代理协议，象屿物流为华泰重工代理出'}, {'IsSh

In [ ]:
with open("StopEvent.json", "w") as f:
    json.dump(result, f, ensure_ascii=False, indent=2)

In [48]:
import pandas as pd
shut_down_table=pd.DataFrame(result)
import re

In [ ]:
stat_table=[]
for i, sub_table in shut_down_table.loc[shut_down_table["IsShutDown"]==True].groupby("ReasonForShutDown"):
    print(i, ": ", len(sub_table))
    stat_table.append({"ReasonForShutDown": i, "count": len(sub_table)})

二者出现竞争关系 :  25
供应商价格过高 :  12
供应商变更 :  1
供应商稳定性不足 :  65
供应商质量不达标 :  152
信息不足 :  2
合同正常到期 :  1
合同自然到期 :  2
市场需求变化 :  1
意外灾害 :  7
技术路线调整 :  40
政策变化与国际制裁 :  219
未明示，可能为供应商稳定性不足或其他经营考量 :  1
法律纠纷 :  106
经营战略调整 :  258
自主研发或国产替代 :  44


In [53]:
pd.DataFrame(stat_table).to_excel(r"result\Stopped_Supply.xlsx")

In [67]:
def get_top_n_frequent_values(df, column_name, n):
    """
    返回指定列中出现次数前 n 的值及其出现次数，按出现次数降序排列。

    参数:
        df (pd.DataFrame): 输入的 DataFrame。
        column_name (str): 要统计的列名。
        n (int): 需要返回的前 n 个值。

    返回:
        pd.DataFrame: 包含两列：'Value'（值）和 'Count'（出现次数），按出现次数降序排列。
    """
    # 统计指定列的值出现频次
    value_counts = df[column_name].value_counts()
    
    # 取前 n 个值
    top_n_values = value_counts.head(n).reset_index()
    
    # 重命名列
    top_n_values.columns = ['Value', 'Count']
    
    return top_n_values

political_shut_table=shut_down_table.loc[shut_down_table["IsShutDown"]].loc[shut_down_table["ReasonForShutDown"]=="政策变化与国际制裁"]
get_top_n_frequent_values(political_shut_table, "supplier", 10).to_excel("result\Stopped_Suppier.xlsx")


In [68]:
get_top_n_frequent_values(political_shut_table, "customer", 10).to_excel("result\Stopped_Costomer.xlsx")


In [ ]:
reason_for_stop=[]

for record in tqdm(stop_event):
    supplier=record["n"].get("name", "")
    supplier_country=record["n"].get("country", "")
    original_content=record["r"].get("original_content", "")
    if "process" in record["r"]:
        analysing_prcess=record["r"].get("process", "")
    elif "analysing_prcess" in record["r"]:
        analysing_prcess=record["r"].get("analysing_prcess", "")
    customer=record["m"].get("name", "")
    customer_country=record["m"].get("country", "")
    
    if original_content:
        ai_response=ask_qwen(
            prompt_text=f'''
                请看一段关于{supplier}和{customer}的文本，严格遵照文本，结合你的逻辑分析回答问题
                {original_content}
                1. 请根据它来回答你认为上述描述中是否存在供货关系终止的情况？请先给出你的分析，在IsShutDown_Analyzation处进行分析。如果存在，请在IsShutDown处填写True，如果不存在，请填写False
                2. 如果存在供货关系终止的情况，请在ReasonForShutDown_Analyzation处分析终止发生的原因，然后ReasonForShutDown处用一个短语概括这一原因。可以选择的原因短语包括：政策变化与国际制裁、自主研发或国产替代、二者出现竞争关系、经营战略调整、技术路线调整、法律纠纷、意外灾害、供应商质量不达标、供应商价格过高、供应商稳定性不足等等。从这些当中仅选择一个你认为最符合的。如果没有合适的，可以填写你认为更符合的。
                你需要以如下的json格式回答问题：
                {{
                    "IsShutDown_Analyzation": str,
                    "IsShutDown": bool,
                    "ReasonForShutDown_Analyzation": str
                    "ReasonForShutDown": str
                }}
                务必基于原文
            ''',
            system_instruction="你是一个行业研究人员，擅长分析商业活动中的因果关系，用json回答问题",
            mode="json",
            model="qwen-max"
        )
        
        try:
            answer_dict=json.loads(reason_for_stop)
            answer_dict.update({
                "supplier": supplier,
                "supplier_country": supplier_country,
                "customer": customer,
                "customer_country": customer_country,
                "original_content": original_content
            })
            
            reason_for_stop.append(answer_dict)
        except Exception as e:
            print(e)
            

{
    "IsShutDown_Analyzation": "根据提供的文本，可以得知中兴通讯（ZTE）被禁止从美国购买关键组件七年，这直接影响了它与美国半导体公司高通之间的供货关系。因为ZTE依赖于像高通这样的美国企业提供的芯片来制造其智能手机和网络设备，所以该禁令导致了两者之间至少在接下来的七年内无法维持原有的供货合作关系。",
    "IsShutDown": true,
    "ReasonForShutDown_Analyzation": "此事件背后的原因是政策层面的变化，具体表现为美国对特定中国企业的制裁措施。这种政府级别的干预直接切断了中兴通讯获取必要进口零件的渠道，从而影响到了其与包括高通在内的多家供应商的合作。",
    "ReasonForShutDown": "政策变化与国际制裁"
}
{
    "IsShutDown_Analyzation": "根据提供的文本，存在一个明显的供货关系终止情况。文中提到ZTE被禁止从美国购买关键组件长达七年，这直接导致了ZTE与美国供应商如高通之间的供应链中断。此外，中国政府采购计划中开始加入本地制造的CPU服务器，意味着之前依赖于Intel芯片供应的情况发生了改变，显示出一种向国产替代品转移的趋势。",
    "IsShutDown": true,
    "ReasonForShutDown_Analyzation": "供货关系终止的主要原因是由于美国对特定中国企业实施了出口禁令（以ZTE为例），迫使中国更加重视本土半导体产业的发展。同时，这也反映了政策层面鼓励使用本国生产的硬件设备来减少对外部技术的依赖。因此，这种变化不仅是因为国际制裁措施，也是国家推动自主研发和国产化战略的结果。",
    "ReasonForShutDown": "政策变化与国际制裁、自主研发或国产替代"
}
{
    "IsShutDown_Analyzation": "从提供的信息来看，CHPT原本是作为基板的购买方（即取引先），但现在已经成为了一个竞争者（競合メーカー）。这表明CHPT可能已经开始了自我生产或找到了其他供应商来满足其对于基板的需求，从而减少了与之前供应商之间的交易。不过，文本中并没有明确指出任何一方正式结束了供货关系；相反地，它暗示了由于CHPT的角色转变导致了自然的业务调整。",
   

KeyboardInterrupt: 

In [28]:
original_content

''

2